In [1]:
import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf

from read_record import Record,RecordReader
from scanning_window import scan_record
from collections import Counter

In [2]:
folder_path="D:/CPSC2021/Training_set_I"
file_name="data_0_1"

In [3]:
reading_record=RecordReader.read(folder_path,file_name,0,0,None)

In [4]:
# Get the keys of the object
keys=reading_record.__dict__.keys()

# Convert the keys to a list if you need
keys_list = list(keys)

# Print the keys
print(keys_list)

['_Record__parent', '_Record__signal', '_Record__symbol', '_Record__aux', '_Record__sample', '_Record__label', '_Record__sf', '_Record__collection']


In [5]:
ecg_signal=reading_record._Record__signal
duration=len(ecg_signal)/reading_record._Record__sf
duration

1041.905

In [6]:
record_rhythm=reading_record._Record__aux
Counter(record_rhythm)

Counter({'': 1266})

In [7]:
record_label=reading_record._Record__label
record_label

'non atrial fibrillation'

In [8]:
ecg_symbol=reading_record._Record__symbol
Counter(ecg_symbol)

Counter({'N': 1265, 'V': 1})

In [9]:
ecg_sample=reading_record._Record__sample
ecg_sample

array([    30,    181,    337, ..., 208033, 208194, 208352], dtype=int64)

In [10]:
ecg_symbol[72]

'N'

In [11]:
list(Counter(reading_record._Record__aux).keys())==['']

True

In [12]:
get_interval=reading_record.get_valid_rhythm_interval(duration=30)
get_interval


[]

In [13]:
from scanning_window import scan_record
test_scan=scan_record(reading_record,window_width=30)


left_end: 0
right_end: 6000
window_width: 30
left_end: 240
right_end: 6240
window_width: 30
left_end: 480
right_end: 6480
window_width: 30
left_end: 720
right_end: 6720
window_width: 30
left_end: 960
right_end: 6960
window_width: 30
left_end: 1200
right_end: 7200
window_width: 30
left_end: 1440
right_end: 7440
window_width: 30
left_end: 1680
right_end: 7680
window_width: 30
left_end: 1920
right_end: 7920
window_width: 30
left_end: 2160
right_end: 8160
window_width: 30
left_end: 2400
right_end: 8400
window_width: 30
left_end: 2640
right_end: 8640
window_width: 30
left_end: 2880
right_end: 8880
window_width: 30
left_end: 3120
right_end: 9120
window_width: 30
left_end: 3360
right_end: 9360
window_width: 30
left_end: 3600
right_end: 9600
window_width: 30
left_end: 3840
right_end: 9840
window_width: 30
left_end: 4080
right_end: 10080
window_width: 30
left_end: 4320
right_end: 10320
window_width: 30
left_end: 4560
right_end: 10560
window_width: 30
left_end: 4800
right_end: 10800
window_width

In [17]:
test_scan[0]

0     -0.007543
1      0.058197
2     -0.032425
3     -0.005526
4     -0.123386
         ...   
839         NaN
840         NaN
841         NaN
842         NaN
843         NaN
Name: 0, Length: 1688, dtype: float64

In [ ]:
signal_data_frame=test_scan.signals
signal_data_frame

In [24]:
import tensorflow as tf

def classify_with_model(model_path, beat_image_path):
    # Load the SavedModel
    model = tf.saved_model.load(model_path)

    # Print the signatures
    print(list(model.signatures.keys()))

    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(beat_image_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.expand_dims(img, 0)

    # Perform inference using the serving_default signature
    inference_func = model.signatures['serving_default']
    output = inference_func(tf.constant(img))
    
    # Print the keys of the output dictionary
    print(output.keys())

    # Process the output
    predictions = output['output_0']  # Adjust 'output' based on your model's signature
    class_idx = tf.argmax(predictions[0])

    # Map the index to the corresponding class label
    class_labels = ["AF", "NSR", "PAC", "PVC"]
    class_label = class_labels[class_idx]

    return class_label


In [29]:
def organize_images(model_path, image_folder_path):
    '''
    This function organizes images from the specified image folder path into sub-folders based on
    their classification predicted by a given model. 
    It copies images to respective class folders and renames them by appending the sub-folder name 
    to the original file name.

Parameters:
- model_path (str): The path to the saved model used for image classification.
- image_folder_path (str): The path to the folder containing images to be organized.

Returns:
None

Functionality:
1. Initializes a dictionary `class_folders` to store paths for class folders based on class labels.
2. Creates directories for each class label (AF, NSR, PAC, PVC) in the image folder path if they do not exist already.
3. Traverses through the image folder and its sub-folders using `os.walk`.
4. For each image file found (.jpg, .jpeg, .png), it:
   - Determines its classification using the provided model and `classify_with_model` function.
   - Copies the image to the corresponding class folder.
   - Renames the copied image by appending the sub-folder name to the original file name.

    '''
    class_labels = ["AF", "NSR", "PAC", "PVC"]
    # Create directories to store images for each class
    class_folders = {}
    for label in class_labels:
        class_folders[label.lower()] = os.path.join(image_folder_path, label.lower())
        os.makedirs(class_folders[label.lower()], exist_ok=True)

    for root, _, files in os.walk(image_folder_path):
        for image_file in files:
            if image_file.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, image_file)
                classification = classify_with_model(model_path, image_path)
                # Move the image to the respective class folder
                dest_folder = class_folders[classification.lower()]
                dest_file_name = os.path.join(dest_folder, os.path.basename(root) + '_' + image_file)
                shutil.copy(image_path, dest_file_name)



In [30]:
# Example usage
model_path = "D:\\GPU\\WithD5\\ecgmv3net1519_saved"
image_folder_path = "D:\\RA\\Project\\Working-with-CPSC-2021-Dataset\\PAC"
organize_images(model_path, image_folder_path)

['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['output_0'])
['serving_default']
dict_keys(['

KeyboardInterrupt: 